# Stats analysis
This notebook read from dumped scores through different approach and complete stats tests

## import 

In [1]:
import joblib
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
from ALSampler import summarize
from itertools import chain

In [167]:
m_scores=joblib.load(r'..\data\vb_sampling\ner_crf_scores.joblib')

In [168]:
r_scores=joblib.load(r'..\data\vb_sampling\ner_rand_scores.joblib')

In [169]:
vmax_scores=joblib.load(r'..\data\vb_sampling\ner_vbmax_scores.joblib')

In [170]:
vmin_scores=joblib.load(r'..\data\vb_sampling\ner_vb_scores.joblib')

In [58]:
name

['precision_m',
 'precision_v',
 'precision_pv',
 'recall_m',
 'recall_v',
 'recall_pv',
 'f1_m',
 'f1_v',
 'f1_pv']

In [60]:
name1='m'
name2='v'
measures=['precision','recall','f1'] 
cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
results={col:[] for col in cols}

In [61]:
results

{'precision_m': [],
 'precision_v': [],
 'precision_pv': [],
 'recall_m': [],
 'recall_v': [],
 'recall_pv': [],
 'f1_m': [],
 'f1_v': [],
 'f1_pv': []}

In [154]:
def compute_stats(scores1, scores2, name1, name2, runs=3, rounds=9):
    measures=['precision','recall','f1'] 
    cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
    results={col:[] for col in cols}
    for ri in range(rounds):
        single_round_scores={name1:{}, name2:{}}
        for m in measures:             
            # print('s1',len(scores1), len(scores1[0]), len(scores1[0][0][m]))
            # print('s2',len(scores2), len(scores2[0]), len(scores2[0][0][m]))
            single_round_scores[name1][m]=list(chain.from_iterable([scores1[u][ri][m] for u in range(runs)]))
            single_round_scores[name2][m]=list(chain.from_iterable([scores2[u][ri][m] for u in range(runs)]))
            results[m+'_'+name1].append(np.mean(single_round_scores[name1][m]))
            results[m+'_'+name2].append(np.mean(single_round_scores[name2][m]))
            ttres=ttest_ind(single_round_scores[name1][m], single_round_scores[name2][m], usevar='unequal')
            # ttres=DescrStatsW.ttest_mean(np.array(single_round_scores[name1][m])-np.array(single_round_scores[name2][m]))
            results[m+'_pv'].append(ttres[1])
    return pd.DataFrame(results)

In [155]:
list(range(3))

[0, 1, 2]

In [165]:
results=compute_stats(m_scores, vmin_scores, 'm', 'vm')

In [166]:
results

,precision_m,precision_vm,precision_pv,recall_m,recall_vm,recall_pv,f1_m,f1_vm,f1_pv
0,0.94354,0.94354,1.00000,0.77924,0.77924,1.00000,0.85355,0.85355,1.00000
1,0.94547,0.94068,0.00000,0.80927,0.81236,0.00000,0.87208,0.87182,0.04947
2,0.94514,0.93962,0.00000,0.82716,0.82583,0.00000,0.88222,0.87906,0.00000
3,0.94608,0.94351,0.00000,0.83743,0.83537,0.00000,0.88844,0.88615,0.00000
4,0.94484,0.94444,0.00008,0.84409,0.84421,0.48890,0.89163,0.89151,0.33418
5,0.94443,0.94203,0.00000,0.85274,0.85270,0.79858,0.89624,0.89514,0.00000
6,0.94720,0.94560,0.00000,0.85277,0.85293,0.26613,0.89751,0.89688,0.00000
7,0.94515,0.94543,0.00291,0.85741,0.85597,0.00000,0.89915,0.89848,0.00000
8,0.94809,0.94493,0.00000,0.85707,0.85894,0.00000,0.90028,0.89988,0.00003


In [160]:
results=compute_stats(m_scores, r_scores, 'm', 'r')
results

,precision_m,precision_r,precision_pv,recall_m,recall_r,recall_pv,f1_m,f1_r,f1_pv
0,0.94354,0.94354,1.00000,0.77924,0.77924,1.00000,0.85355,0.85355,1.00000
1,0.94547,0.94329,0.00000,0.80927,0.80892,0.08208,0.87208,0.87095,0.00000
2,0.94514,0.94297,0.00000,0.82716,0.82462,0.00000,0.88222,0.87983,0.00000
3,0.94608,0.94331,0.00000,0.83743,0.83339,0.00000,0.88844,0.88495,0.00000
4,0.94484,0.94357,0.00000,0.84409,0.84271,0.00000,0.89163,0.89029,0.00000
5,0.94443,0.94570,0.00000,0.85274,0.84489,0.00000,0.89624,0.89245,0.00000
6,0.94720,0.94593,0.00000,0.85277,0.84981,0.00000,0.89751,0.89530,0.00000
7,0.94515,0.94461,0.00000,0.85741,0.85417,0.00000,0.89915,0.89711,0.00000
8,0.94809,0.94567,0.00000,0.85707,0.85806,0.00000,0.90028,0.89974,0.00000


In [163]:
results=compute_stats(r_scores, vmin_scores, 'r', 'v')
results

,precision_r,precision_v,precision_pv,recall_r,recall_v,recall_pv,f1_r,f1_v,f1_pv
0,0.94354,0.94354,1.00000,0.77924,0.77924,1.00000,0.85355,0.85355,1.00000
1,0.94329,0.93854,0.00000,0.80892,0.81437,0.00000,0.87095,0.87206,0.00000
2,0.94297,0.94266,0.00071,0.82462,0.82448,0.44366,0.87983,0.87962,0.07490
3,0.94331,0.94431,0.00000,0.83339,0.83584,0.00000,0.88495,0.88677,0.00000
4,0.94357,0.94177,0.00000,0.84271,0.84811,0.00000,0.89029,0.89249,0.00000
5,0.94570,0.94449,0.00000,0.84489,0.85013,0.00000,0.89245,0.89483,0.00000
6,0.94593,0.94552,0.00000,0.84981,0.85329,0.00000,0.89530,0.89704,0.00000
7,0.94461,0.94501,0.00002,0.85417,0.85861,0.00000,0.89711,0.89974,0.00000
8,0.94567,0.94637,0.00000,0.85806,0.85939,0.00000,0.89974,0.90079,0.00000
